In [2]:
#!pip install --upgrade ~/Development/repos/tradeframework
#!pip install --upgrade ~/Development/repos/quantutils

# import utilities
from MIPriceAggregator.api.aggregator import MarketDataAggregator
import quantutils.dataset.pipeline as ppl

#import tradeframework
from tradeframework.api import Derivative, Optimizer, Model, TradeEnvironment, TradeEngine, Asset
from tradeframework.environments import SandboxEnvironment, ZiplineEnvironment
from tradeframework.models import PreOpenMomentum
import tradeframework.operations.trader as trader
import tradeframework.operations.utils as utils

import pandas as pd
import numpy as np
import quantutils.core.timeseries as tsUtils

import matplotlib
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12.0, 6.0)


In [2]:
data_config = [
    {
        "ID": "MDS",
        "class": "MDSConnector",
        "opts": {
            "remote":True,
            "location":"http://pricestore.192.168.1.203.nip.io"
        },
        "timezone":"UTC",
        "markets": [
            {
                "ID": "DOW",
                "sources": [
                    {
                        "ID": "D&J-IND",
                        "sample_unit": "5min"
                    }
                ]
            }
            
        ]

    }
]

## Get Market Data
aggregator = MarketDataAggregator(data_config)
   
#start = "2013-01-01"
#end = "2020-07-11"

start = "2013-01-01"
end = "2013-07-10 18:00"

ts = aggregator.getData( "DOW", "H", start, end, debug=True)
ts = ts.reset_index().set_index("Date_Time")[["Open", "High", "Low", "Close"]]
ts = ppl.removeNaNs(ts)
ts = ts.tz_convert("US/Eastern", level=0)


Resampling to 5min periods
Resampling to H periods
Resampling to H periods


In [3]:
env = SandboxEnvironment("TradeFair", "US/Eastern")

#opt = env.createOptimizer("KellyOptimizer", "KellyOpt", opts={"window":40*24, "weight":.01})
p = env.createPortfolio("MyPortfolio", optimizer=env.createOptimizer("EqualWeights", "EqualWeightsOptimizer", opts={"weight":1.0}))
env.setPortfolio(p)

# Non-Weighted Portfolio (Indicator)
#indicatorP = env.createPortfolio("Indicators")
#p.addAsset(indicatorP, weighted=False)
#indicatorP.addAsset(env.createModel( "TrendFollowingIndicator", "TrendFollowing", opts={"start": "15:00", "end": "16:00", "barOnly": True} ))
#indicatorP.addAsset(env.createModel( "MA20", "MovingAverage", opts={"ma1_win":20} ))
#indicatorP.addAsset(env.createModel( "MAXOver", "MovingAverageXOver", opts={"ma1_fast":50, "ma2_slow":200} ))
#indicatorP.addAsset(env.createModel( "MADailyX", "MovingAverageResampleXOver", opts={"ma1_fast":1, "ma2_slow":20, "src_sample":"H", "tgt_sample":"W"} ))
#indicatorP.addAsset(env.createModel( "StochOSC1", "StochasticOscXOver", opts={"window":14, "src_sample":"H", "tgt_sample":"W-MON" } ))
#indicatorP.addAsset(env.createModel( "StochOSC2", "StochasticOscXOver", opts={"window":14, "src_sample":"H", "tgt_sample":"D"} ))
#indicatorP.addAsset(env.createModel( "StochOSC3", "StochasticOscXOver", opts={"window":14, "src_sample":"H", "tgt_sample":"H"} ))
#indicatorP.addAsset(env.createModel( "BuyAndHoldIndicator", "BuyAndHold" ))

#p.addAsset(MovingAverage("MA20", env, 14))

# Combination Model (using Indicators)
#p.addAsset(env.createModel("CombiTrendAndBuy", "CombinationModel", opts={"modelList": [ "StochOSC1", "StochOSC2", "BuyAndHoldIndicator"]}))

# Custom Models
#p.addAsset(env.createModel( "D1-PreOpenMomentum", "PreOpenMomentum" ))
#p.addAsset(env.createModel( "MA20", "MovingAverage", opts={"ma1_win":20} ))
#p.addAsset(env.createModel( "MAX", "MovingAverageXOver", opts={"ma1_fast":50, "ma2_slow":200} ))
#p.addAsset(env.createModel( "MADaily", "MovingAverageResample", opts={"ma1_win":200, "src_sample":"H", "tgt_sample":"B"} ))
#p.addAsset(env.createModel( "MADailyX", "MovingAverageResampleXOver", opts={"ma1_fast":1, "ma2_slow":12, "src_sample":"H", "tgt_sample":"MS"} ))
#p.addAsset(env.createModel( "MADailyX2", "MovingAverageResampleXOver", opts={"ma1_fast":4, "ma2_slow":52, "src_sample":"H", "tgt_sample":"W-MON"} ))
#p.addAsset(env.createModel( "MADailyX2", "MovingAverageResampleXOver", opts={"ma1_fast":50, "ma2_slow":200, "src_sample":"H", "tgt_sample":"D"} ))
#p.addAsset(env.createModel( "StochOSC", "StochasticOscXOver", opts={"window":9, "src_sample":"D", "tgt_sample":"W-MON"} ))

# All hours
#p.addAsset(env.createModel( "D2-BuyAndHold", "BuyAndHold"))

# Market Opening hours only
#p.addAsset(env.createModel( "D3-BuyAndHold7", "BuyAndHold", opts={"start":"09:00", "end":"16:00"}))

# Individual hours
#p.addAsset(env.createModel( "BuyAndHold", "D3-BuyAndHold6", args=("10:00", "11:00")))
#p.addAsset(env.createModel( "BuyAndHold", "D3-BuyAndHold5", args=("11:00", "12:00")))
#p.addAsset(env.createModel( "BuyAndHold", "D3-BuyAndHold4", args=("12:00", "13:00")))
#p.addAsset(env.createModel( "BuyAndHold", "D3-BuyAndHold3", args=("13:00", "14:00")))
#p.addAsset(env.createModel( "BuyAndHold", "D3-BuyAndHold2", args=("14:00", "15:00")))
#p.addAsset(env.createModel( "BuyAndHold", "D3-BuyAndHold1", args=("15:00", "16:00")))
p.addAsset(env.createModel("TestModel", "TrendFollowing", opts={"start": "15:00", "end": "16:00", "barOnly": True}))

# ML models
threshold = 0
#p.addAsset(env.createModel( "MarketInsightsV2Model", "ML-BasicBootstrap_DOW", args=(CredentialsStore(), '4234f0f1b6fcc17f6458696a6cdf5101', '4892e256e17f4fad775119304ee7e6d1', threshold)))
#p.addAsset(env.createModel( "MarketInsightsV2Model", "ML-BasicBootstrap_SPY", args=(CredentialsStore(), '3231bbe5eb2ab84eb54c9b64a8dcea55', '4892e256e17f4fad775119304ee7e6d1', threshold)))
#p.addAsset(env.createModel( "MarketInsightsV2Model", "ML-BasicBoosting_DOW", args=(CredentialsStore(), '4234f0f1b6fcc17f6458696a6cdf5101', '83c0ff9eeb24c87dbb0af766ab59ef75', threshold)))
#p.addAsset(env.createModel( "MarketInsightsV2Model", "ML-BasicBoosting_SPY", args=(CredentialsStore(), '3231bbe5eb2ab84eb54c9b64a8dcea55', '83c0ff9eeb24c87dbb0af766ab59ef75', threshold)))
#p.addAsset(env.createModel( "MarketInsightsV2Model", "ML-OneStepAhead_Boosting_SPY", args=(CredentialsStore(), '3231bbe5eb2ab84eb54c9b64a8dcea55', 'f2efe64bbc39ad0880337120f32ca12b', threshold)))
#p.addAsset(env.createModel( "MarketInsightsV2Model", "ML-OneStepAhead_Boosting_DOW", args=(CredentialsStore(), '4234f0f1b6fcc17f6458696a6cdf5101', 'f2efe64bbc39ad0880337120f32ca12b', threshold)))
#p.addAsset(env.createModel( "MarketInsightsV2Model", "ML-OneStepAhead_Bootstrap_SPY", args=(CredentialsStore(), '3231bbe5eb2ab84eb54c9b64a8dcea55', '078df5a1afbaa2290ee93b4a562e3898', threshold)))
#p.addAsset(env.createModel( "MarketInsightsV2Model", "ML-OneStepAhead_Bootstrap_DOW", args=(CredentialsStore(), '4234f0f1b6fcc17f6458696a6cdf5101', '078df5a1afbaa2290ee93b4a562e3898', threshold)))

#Sandbox
#p.addAsset(env.createModel( "D3-BuyAndHold7", "BuyAndHold"))
#p.addAsset(env.createModel( "D3-BuyAndHold7", "SellAndHold", opts={"start":"09:00", "end":"16:00", "barOnly":"True"}))

# Feed data into the portfolio

#for index in range(0, len(ts)):
#env.handleData(0, ts[:index+1])
dInfo = env.append(Asset("DOW", ts), copy=False) 


In [4]:
trader.getCurrentSignal(p, 10000)

{'timestamp': '2013-07-10T14:00:00-04:00',
 'value': 0.9912554640452014,
 'capital': 10000,
 'target': 'CLOSE',
 'markets': [{'name': 'DOW',
   'price': 15336.06,
   'signal': 'HOLD',
   'amount': 0.0}]}

In [8]:
ohlc = [[17407.46, np.nan, np.nan, np.nan]]
index = pd.DatetimeIndex(['2013-07-10 15:00:00'], tz='US/Eastern') # TODO: Provide UK timezone

trader.predictSignals(env, utils.createAssetFromOHLC(index, ohlc, "DOW"), 10000)

[{'timestamp': '2013-07-10T15:00:00-04:00',
  'value': 0.9912554640452014,
  'capital': 10000,
  'target': 'OPEN',
  'markets': [{'name': 'DOW',
    'price': 17407.46,
    'signal': 'BUY',
    'amount': 0.5694429078367559}]}]

In [7]:
ohlc = [[17407.46, 13411.91, 13377.22, 17387.45]]
index = pd.DatetimeIndex(['2013-07-10 15:00:00'], tz='US/Eastern') # TODO: Provide UK timezone

trader.predictSignals(env, utils.createAssetFromOHLC(index, ohlc, "DOW"), 10000)

[{'timestamp': '2013-07-10T15:00:00-04:00',
  'value': 0.9901160087866201,
  'capital': 10000,
  'target': 'CLOSE',
  'markets': [{'name': 'DOW',
    'price': 17387.45,
    'signal': 'SELL',
    'amount': 0.5694429078367559}]}]

In [9]:
possibles = []

for i in range(-10,10):
    ohlc = [[15274.38 + i, np.nan, np.nan, np.nan]]
    index = pd.DatetimeIndex(['2013-07-10 15:00:00'], tz='US/Eastern')
    possibles.append(utils.createAssetFromOHLC(index, ohlc, "DOW"))
    
trader.predictSignals(env, possibles, 10000)

[{'timestamp': '2013-07-10T15:00:00-04:00',
  'value': 0.9912554640452014,
  'capital': 10000,
  'target': 'OPEN',
  'markets': [{'name': 'DOW',
    'price': 15264.38,
    'signal': 'SELL',
    'amount': 0.6493912389793765}]},
 {'timestamp': '2013-07-10T15:00:00-04:00',
  'value': 0.9912554640452014,
  'capital': 10000,
  'target': 'OPEN',
  'markets': [{'name': 'DOW',
    'price': 15265.38,
    'signal': 'SELL',
    'amount': 0.6493486988500787}]},
 {'timestamp': '2013-07-10T15:00:00-04:00',
  'value': 0.9912554640452014,
  'capital': 10000,
  'target': 'OPEN',
  'markets': [{'name': 'DOW',
    'price': 15266.38,
    'signal': 'SELL',
    'amount': 0.6493061642938283}]},
 {'timestamp': '2013-07-10T15:00:00-04:00',
  'value': 0.9912554640452014,
  'capital': 10000,
  'target': 'OPEN',
  'markets': [{'name': 'DOW',
    'price': 15267.38,
    'signal': 'SELL',
    'amount': 0.6492636353095301}]},
 {'timestamp': '2013-07-10T15:00:00-04:00',
  'value': 0.9912554640452014,
  'capital': 1000

In [10]:
trader.getTradingInfo(p,1, summary=False)


MyPortfolio                 DOW            \
                              Capital               Price             
                                 Open     Close      Open     Close   
Date_Time                                                             
2013-01-01 00:00:00-05:00    1.000000  1.000000  13104.14  13104.14   
2013-01-01 17:00:00-05:00    1.000000  1.000000  13104.14  13104.14   
2013-01-02 00:00:00-05:00    1.000000  1.000000  13104.14  13104.14   
2013-01-02 09:00:00-05:00    1.000000  1.000000  13104.30  13366.42   
2013-01-02 10:00:00-05:00    1.000000  1.000000  13366.48  13345.45   
...                               ...       ...       ...       ...   
2013-07-10 10:00:00-04:00    0.991255  0.991255  15317.72  15281.24   
2013-07-10 11:00:00-04:00    0.991255  0.991255  15281.22  15272.60   
2013-07-10 12:00:00-04:00    0.991255  0.991255  15272.51  15273.77   
2013-07-10 13:00:00-04:00    0.991255  0.991255  15273.79  15274.35   
2013-07-10 14:00:00-04:00    0.991255  0.991255  15274.38  15336.06   

                                                      
                          Allocation      Trade       
                                 bar  gap   bar  gap  
Date_Time                                             
2013-01-01 00:00:00-05:00        0.0  0.0   0.0  0.0  
2013-01-01 17:00:00-05:00        0.0  0.0   0.0  0.0  
2013-01-02 00:00:00-05:00        0.0  0.0   0.0  0.0  
2013-01-02 09:00:00-05:00        0.0  0.0   0.0  0.0  
2013-01-02 10:00:00-05:00        0.0  0.0   0.0  0.0  
...                              ...  ...   ...  ...  
2013-07-10 10:00:00-04:00        0.0  0.0   0.0  0.0  
2013-07-10 11:00:00-04:00        0.0  0.0   0.0  0.0  
2013-07-10 12:00:00-04:00        0.0  0.0   0.0  0.0  
2013-07-10 13:00:00-04:00        0.0  0.0   0.0  0.0  
2013-07-10 14:00:00-04:00        0.0  0.0   0.0  0.0  

[1459 rows x 8 columns]

In [11]:
trader.getTradingInfo(p,10000, unitAllocations=False)

MyPortfolio                     DOW            \
                                Capital                   Price             
                                   Open         Close      Open     Close   
Date_Time                                                                   
2013-01-02 15:00:00-05:00  10000.000000  10047.532859  13331.83  13395.20   
2013-01-03 15:00:00-05:00  10047.532859  10069.192226  13406.38  13377.48   
2013-01-04 15:00:00-05:00  10069.192226  10077.506245  13419.10  13430.18   
2013-01-07 15:00:00-05:00  10077.506245  10078.553059  13381.30  13382.69   
2013-01-08 15:00:00-05:00  10078.553059  10067.892634  13311.48  13325.56   
...                                 ...           ...       ...       ...   
2013-07-01 15:00:00-04:00   9978.541579   9935.073617  15038.53  14973.02   
2013-07-02 15:00:00-04:00   9935.073617   9915.595720  14909.32  14938.55   
2013-07-05 15:00:00-04:00   9915.595720   9928.240284  15111.12  15130.39   
2013-07-08 15:00:00-04:00   9928.240284   9918.474306  15239.06  15224.07   
2013-07-09 15:00:00-04:00   9918.474306   9912.554640  15297.43  15306.56   

                                                               
                          Allocation          Trade            
                                 bar  gap       bar       gap  
Date_Time                                                      
2013-01-02 15:00:00-05:00   0.750085  0.0  0.750085 -0.750085  
2013-01-03 15:00:00-05:00  -0.753021  0.0 -0.753021  0.753021  
2013-01-04 15:00:00-05:00   0.755555  0.0  0.755555 -0.755555  
2013-01-07 15:00:00-05:00   0.758941  0.0  0.758941 -0.758941  
2013-01-08 15:00:00-05:00  -0.763080  0.0 -0.763080  0.763080  
...                              ...  ...       ...       ...  
2013-07-01 15:00:00-04:00   0.662108  0.0  0.662108 -0.662108  
2013-07-02 15:00:00-04:00  -0.662040  0.0 -0.662040  0.662040  
2013-07-05 15:00:00-04:00   0.650640  0.0  0.650640 -0.650640  
2013-07-08 15:00:00-04:00   0.646824  0.0  0.646824 -0.646824  
2013-07-09 15:00:00-04:00  -0.643089  0.0 -0.643089  0.643089  

[129 rows x 8 columns]

In [14]:
trader.getTradingInfo(p,10000)

MyPortfolio                     DOW            \
                                Capital                   Price             
                                   Open         Close      Open     Close   
Date_Time                                                                   
2013-01-02 15:00:00-05:00  10000.000000  10047.532859  13331.83  13395.20   
2013-01-03 15:00:00-05:00  10047.532859  10069.192226  13406.38  13377.48   
2013-01-04 15:00:00-05:00  10069.192226  10077.506245  13419.10  13430.18   
2013-01-07 15:00:00-05:00  10077.506245  10078.553059  13381.30  13382.69   
2013-01-08 15:00:00-05:00  10078.553059  10067.892634  13311.48  13325.56   
2013-01-09 15:00:00-05:00  10067.892634  10048.245526  13364.34  13390.42   
2013-01-10 15:00:00-05:00  10048.245526  10054.749351  13456.73  13465.44   
2013-01-11 15:00:00-05:00  10054.749351  10057.815431  13478.13  13482.24   
2013-01-14 15:00:00-05:00  10057.815431  10053.877297  13510.42  13505.13   
2013-01-15 15:00:00-05:00  10053.877297  10067.993231  13518.24  13537.22   
2013-01-16 15:00:00-05:00  10067.993231  10063.828214  13512.57  13506.98   
2013-01-17 15:00:00-05:00  10063.828214  10075.117822  13620.96  13605.68   
2013-01-18 15:00:00-05:00  10075.117822  10098.765466  13608.09  13640.03   
2013-01-22 15:00:00-05:00  10098.765466  10102.443705  13700.26  13705.25   
2013-01-23 15:00:00-05:00  10102.443705  10092.796070  13790.86  13777.69   
2013-01-24 15:00:00-05:00  10092.796070  10095.920698  13824.74  13829.02   
2013-01-25 15:00:00-05:00  10095.920698  10088.309625  13874.96  13885.42   
2013-01-28 15:00:00-05:00  10088.309625  10075.778068  13902.91  13885.64   
2013-01-29 15:00:00-05:00  10075.778068  10068.807576  13948.98  13958.63   
2013-01-30 15:00:00-05:00  10068.807576  10086.985581  13930.60  13905.45   
2013-01-31 15:00:00-05:00  10086.985581  10104.442086  13896.94  13872.89   
2013-02-01 15:00:00-05:00  10104.442086  10100.753883  13999.69  14004.80   
2013-02-04 15:00:00-05:00  10100.753883  10071.841824  13915.06  13875.23   
2013-02-05 15:00:00-05:00  10071.841824  10050.006929  14004.24  13973.88   
2013-02-06 15:00:00-05:00  10050.006929  10064.214632  13963.35  13983.09   
2013-02-07 15:00:00-05:00  10064.214632  10062.150123  13942.13  13939.27   
2013-02-08 15:00:00-05:00  10062.150123  10062.315616  13984.29  13984.06   
2013-02-11 15:00:00-05:00  10062.315616  10052.266759  13958.67  13972.61   
2013-02-12 15:00:00-05:00  10052.266759  10060.641928  14030.88  14019.19   
2013-02-13 15:00:00-05:00  10060.641928  10040.285963  13952.27  13980.50   
...                                 ...           ...       ...       ...   
2013-05-28 15:00:00-04:00   9882.888629   9885.267509  15412.93  15416.64   
2013-05-29 15:00:00-04:00   9885.267509   9889.294539  15317.51  15311.27   
2013-05-30 15:00:00-04:00   9889.294539   9849.600043  15391.57  15329.79   
2013-05-31 15:00:00-04:00   9849.600043   9910.931762  15238.91  15144.02   
2013-06-03 15:00:00-04:00   9910.931762   9947.709875  15193.23  15249.61   
2013-06-04 15:00:00-04:00   9947.709875   9972.061811  15143.01  15180.08   
2013-06-05 15:00:00-04:00   9972.061811   9996.124530  14993.70  14957.52   
2013-06-06 15:00:00-04:00   9996.124530  10034.864878  14973.41  15031.44   
2013-06-07 15:00:00-04:00  10034.864878  10059.948213  15210.32  15248.34   
2013-06-10 15:00:00-04:00  10059.948213  10036.930829  15262.09  15227.17   
2013-06-11 15:00:00-04:00  10036.930829   9996.308435  15182.99  15121.54   
2013-06-12 15:00:00-04:00   9996.308435  10004.660861  15008.06  14995.52   
2013-06-13 15:00:00-04:00  10004.660861  10042.316314  15123.05  15179.97   
2013-06-14 15:00:00-04:00  10042.316314  10035.201764  15060.90  15071.57   
2013-06-17 15:00:00-04:00  10035.201764   9999.390877  15126.69  15180.67   
2013-06-18 15:00:00-04:00   9999.390877   9987.418682  15334.60  15316.24   
2013-06-19 15:00:00-04:00   9987.418682  10003.336962  15152.15  15128.00   
2013-06-20 15:00:00-04:0